In [1]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets 
from torchvision import transforms
from torchvision.utils import save_image
from torch.autograd import Variable


import pandas as pd 
import numpy as np
import os 
import glob
np.random.seed(1337)  # for reproducibility
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import torch.optim as optim
import torch
import torch.nn.functional as nn
import torch.autograd as autograd
import torch.optim as optim
import numpy as np
#import matplotlib.pyplot as plt
#import matplotlib.gridspec as gridspec
import os
from torch.autograd import Variable
from sklearn.utils import shuffle
import random
from keras.datasets import mnist
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.utils import np_utils
from keras import backend as K
from keras import optimizers

Using TensorFlow backend.


In [2]:
train_lre = pd.read_csv('/home/satishk/lre2.0/ivectors_csv_revised/train_feat_BNF_h5_07Nov_Shreyas.csv')
#train_afds

In [3]:
#train_lre = train_lre.iloc[1000:2000]

In [4]:
train_lre.head()

,0,1,2,3,4,5,6,7,8,9,...,491,492,493,494,495,496,497,498,499,langid
0,-0.141175,-1.160019,0.891814,1.898842,0.393065,0.983582,-0.559143,-0.761900,0.525598,0.344597,...,1.587687,-0.609223,1.529694,1.677775,-0.388426,1.044584,-1.365691,-0.354752,-0.815562,spa-eur
1,0.971834,0.035743,0.785385,1.328749,-0.043342,1.219254,0.761090,1.069464,0.879541,0.898939,...,-1.208031,0.527433,-0.709180,-1.117489,0.566520,1.642208,-0.703815,0.376027,-1.425985,spa-eur
2,-0.315703,-0.868423,0.619893,1.717784,-0.846024,1.177214,-0.191977,-0.658569,0.529625,0.313590,...,-0.896515,-0.128051,-0.075781,1.307904,-1.443047,2.551083,0.436309,-0.623076,0.194851,spa-eur
3,0.247949,-0.279720,1.320115,1.772152,-0.130394,0.517343,-0.290516,0.406225,-0.833366,1.567592,...,-0.398762,-0.078148,0.168076,-0.272796,-1.102862,0.358274,0.090417,-0.490244,1.819196,spa-eur
4,-0.986578,-0.357548,1.550832,1.400735,0.444350,0.465660,0.209248,1.175847,-1.079963,0.270915,...,0.947946,0.917957,-1.043352,0.414811,0.016396,-0.485602,-0.433002,-0.432070,0.778250,spa-eur


In [5]:
train_lre.groupby(['langid']).size()

langid
ara-acm    1406
ara-apc    3509
ara-ary     919
ara-arz     440
eng-gbr      98
eng-usg    2448
por-brz     444
qsl-pol     587
qsl-rus    1221
spa-car     688
spa-eur     121
spa-lac     898
zho-cmn    3331
zho-nan      95
dtype: int64

In [6]:
#Choosing a single class
train_lre = train_lre.loc[train_lre['langid'] == 'ara-acm']

In [7]:
val_lre = pd.read_csv('/home/satishk/lre2.0/ivectors_csv_revised/dev_feat_BNF_h5_07Nov_Shreyas.csv')


In [8]:
#val_lre = val_lre.iloc[100:300]

In [9]:
val_lre.head()

,segmentid,language_code,data_source,speech_duration,0,1,2,3,4,5,...,491,492,493,494,495,496,497,498,499,uttid
0,lre17_ntrlosgu.sph,ara-acm,mls14,30,1.697234,0.029428,-0.400756,0.513963,-0.939232,1.500797,...,-1.314428,-0.927694,-0.370424,-0.514735,1.290885,0.688205,-0.494330,-0.053206,-1.330860,lre17_ntrlosgu
1,lre17_moxnwuqe.sph,ara-acm,mls14,10,1.648232,-0.053318,-0.562867,1.035870,-1.577741,1.593584,...,-0.929262,-1.301574,2.034934,-0.226545,-0.198926,-0.116174,0.347923,-0.870801,-2.599601,lre17_moxnwuqe
2,lre17_meesvkxz.sph,ara-acm,mls14,3,1.242829,0.675515,-0.371491,0.534970,-0.246783,0.806262,...,0.691336,0.257988,1.058771,1.018635,-1.929319,-0.307404,-0.486431,-2.839053,-2.704527,lre17_meesvkxz
3,lre17_rqmsmzui.sph,ara-acm,mls14,30,1.226681,0.014810,-0.396915,-0.097507,-0.013574,1.087025,...,1.049862,0.285627,2.385587,0.680073,1.500978,1.660566,-0.370672,-0.924109,0.096676,lre17_rqmsmzui
4,lre17_qgszpuyw.sph,ara-acm,mls14,10,1.411728,-0.119300,0.136256,0.030535,-1.029447,1.227100,...,0.155196,-1.030222,2.933880,-1.417872,-0.227513,0.748810,-0.671044,0.595977,1.722917,lre17_qgszpuyw


In [10]:
X_train = train_lre.drop("langid",axis=1)
y_train = train_lre["langid"]
#y_train_uttid = train_lre["uttid"]

In [11]:
X_val = val_lre.drop(["language_code","uttid","segmentid","data_source","speech_duration"],axis=1)
y_val = val_lre["language_code"]
y_val_segmentid = val_lre["segmentid"]

In [12]:
X_train = X_train.append(X_val, ignore_index=True)
y_train = y_train.append(y_val, ignore_index=True)

In [13]:
le = preprocessing.LabelEncoder()
le.fit(y_train)

LabelEncoder()

In [14]:
le.classes_

array(['ara-acm', 'ara-apc', 'ara-ary', 'ara-arz', 'eng-gbr', 'eng-usg',
       'por-brz', 'qsl-pol', 'qsl-rus', 'spa-car', 'spa-eur', 'spa-lac',
       'zho-cmn', 'zho-nan'], dtype=object)

In [15]:
y_train=le.transform(y_train)

In [16]:
le = preprocessing.LabelEncoder()
le.fit(y_val)

LabelEncoder()

In [17]:
le.classes_

array(['ara-acm', 'ara-apc', 'ara-ary', 'ara-arz', 'eng-gbr', 'eng-usg',
       'por-brz', 'qsl-pol', 'qsl-rus', 'spa-car', 'spa-eur', 'spa-lac',
       'zho-cmn', 'zho-nan'], dtype=object)

In [18]:
y_val_labels = le.transform(y_val)

In [19]:
y_val_labels[0:100]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0])

In [20]:
X_train=X_train.values
X_val=X_val.values

In [21]:
nb_classes = 14

In [22]:
def indices_to_one_hot(data, nb_classes):
    """Convert an iterable of indices to one-hot encoded labels."""
    targets = np.array(data).reshape(-1)
    return np.eye(nb_classes)[targets]

In [23]:
Y_train = indices_to_one_hot(y_train, 14)
#Y_test = indices_to_one_hot(y_test, 14)
Y_val = indices_to_one_hot(y_val_labels, 14)

In [24]:
Y_val

array([[ 1.,  0.,  0., ...,  0.,  0.,  0.],
       [ 1.,  0.,  0., ...,  0.,  0.,  0.],
       [ 1.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  1.],
       [ 0.,  0.,  0., ...,  0.,  0.,  1.],
       [ 0.,  0.,  0., ...,  0.,  0.,  1.]])

In [25]:
Y_train

array([[ 1.,  0.,  0., ...,  0.,  0.,  0.],
       [ 1.,  0.,  0., ...,  0.,  0.,  0.],
       [ 1.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  1.],
       [ 0.,  0.,  0., ...,  0.,  0.,  1.],
       [ 0.,  0.,  0., ...,  0.,  0.,  1.]])

In [26]:
Y_val.shape

(3661, 14)

In [27]:
Y_train.shape

(5067, 14)

In [28]:
X_train = X_train.astype('float32')
#X_test = X_test.astype('float32')
X_val = X_val.astype('float32')

#X_train /= 255
#X_test /= 255
#X_val /= 255

print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
#print(X_test.shape[0], 'test samples')
print(X_val.shape[0], 'val samples')


X_train shape: (5067, 500)
5067 train samples
3661 val samples


In [29]:
X_train.shape

(5067, 500)

In [30]:
y_train.shape

(5067,)

In [31]:
#X_train,  y_train = shuffle(X_train, y_train, random_state=0)

In [32]:
X_val.shape

(3661, 500)

In [33]:
y_val_labels

array([ 0,  0,  0, ..., 13, 13, 13])

In [34]:
#X_val,  y_val_labels = shuffle(X_val, y_val_labels, random_state=0)

In [35]:
X_train, X_test, Y_train, y_test = train_test_split(X_train, Y_train, test_size=0.2, random_state=42)

In [36]:
#mnist = input_data.read_data_sets('/home/satishk/depy_04_AUG/MNIST_data', one_hot=True)
mb_size = 256
Z_dim = 100
X_dim = 500 #mnist.train.images.shape[1]
y_dim = 14 #mnist.train.labels.shape[1]
h_dim = 128
cnt = 0
lr = 1e-3


In [37]:
X_dim, y_dim

(500, 14)

In [38]:
import torch.nn as nn

In [39]:
#Implementing Generator as Encoder-Decoder pair inspired from DAGAN base paper

#Encoder should be able to take a batch of input(of dim 500-ivector) and be able to produce its representation r

class Encoder(nn.Module):
    def __init__(self):
        super(Encoder, self).__init__()
        self.fc1 = nn.Linear(500, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 128)
    def forward(self, x):
        return self.fc3(F.relu(self.fc2(F.relu(self.fc1(x)))))
    
#We will use Decoder as Generator    
#Decoder should be able to take input of dim 128(r) and 100(z),concatenate r and z and 
#produce an output od dim 500-ivector    

class Decoder(nn.Module):
    def __init__(self):
        super(Decoder, self).__init__()
        self.fc1 = nn.Linear(228,256 )
        self.fc2 = nn.Linear(256, 512)
        self.fc3 = nn.Linear(512,500)
    def forward(self, x):
        #inputs = torch.cat([z, x], 1)
        return F.sigmoid(self.fc3(F.relu(self.fc2(F.relu(self.fc1(x))))))

#input to Generator alias Decoder is inputs = torch.cat([z, r], 1)
#Where r is the output of encoder given x i.e., representation of x encoded by encoder


In [40]:
#As we defined the Encoder and Decoder we now Implement the Generator 

class Generator(nn.Module):
                                
    def __init__(self):
        super(Generator, self).__init__()
        self.fc1 = Encoder()
        self.fc2 = Decoder()

    def forward(self, z, x):
        #inputs = torch.cat([z, x], 1)
        return self.fc2(torch.cat([z, self.fc1(x)],1))

In [41]:
#Implementation of Discriminator

class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        #self.inputs = torch.cat([z, c], 1)
        self.fc1 = torch.nn.Linear(500, 512)
        self.fc2 = torch.nn.Linear(512,512)
        self.fc3 = torch.nn.Linear(512,1)
        
    def forward(self, x):
        #inputs = torch.cat([X, c], 1)
        return F.sigmoid(self.fc3(F.relu(self.fc2(F.relu(self.fc1(x))))))


In [42]:
G = Generator()
D = Discriminator()

In [43]:
G.modules

<bound method Module.modules of Generator(
  (fc1): Encoder(
    (fc1): Linear(in_features=500, out_features=512)
    (fc2): Linear(in_features=512, out_features=256)
    (fc3): Linear(in_features=256, out_features=128)
  )
  (fc2): Decoder(
    (fc1): Linear(in_features=228, out_features=256)
    (fc2): Linear(in_features=256, out_features=512)
    (fc3): Linear(in_features=512, out_features=500)
  )
)>

In [44]:
D

Discriminator(
  (fc1): Linear(in_features=500, out_features=512)
  (fc2): Linear(in_features=512, out_features=512)
  (fc3): Linear(in_features=512, out_features=1)
)

In [45]:
ones_label = Variable(torch.ones(mb_size))
zeros_label = Variable(torch.zeros(mb_size))
ones_label_fake = Variable(torch.ones(mb_size*2))

In [46]:
learning_rate = 0.0002
betas = (0.5, 0.999)

In [47]:
# Loss function
#criterion = torch.nn.BCELoss()

# Optimizers
G_solver = torch.optim.Adam(G.parameters(), lr=learning_rate, betas=betas)
D_solver = torch.optim.Adam(D.parameters(), lr=learning_rate/2, betas=betas)

In [48]:
len(X_train)

4053

In [49]:
from torch.nn.functional import binary_cross_entropy

In [50]:
batch_size = mb_size
# Start training
for epoch in range(100):
    
    

    print('epoch:',epoch)
    #for i in range(XX_train):
    # Build mini-batch dataset
    #batch_size = images.size(0)
    #images = to_var(images.view(batch_size, -1))

    it=0
    while it+batch_size < len(X_train) :
        

        start= it
        end= it + batch_size


        z = Variable(torch.randn(mb_size, Z_dim))
        X = X_train[start:end]

        #c = Y_train[start:end]
        X = Variable(torch.from_numpy(X))
        #c = Variable(torch.from_numpy(c.astype('float32')))

        # Dicriminator forward-loss-backward-update
        G_sample = G(z, X)
        D_real = D(X)
        D_fake = D(G_sample)

        D_loss_real = binary_cross_entropy(D_real, ones_label)
        D_loss_fake = binary_cross_entropy(D_fake, zeros_label)
        D_loss = D_loss_real + D_loss_fake

        D_loss.backward()
        D_solver.step()

        # Housekeeping - reset gradient
        D.zero_grad()

        # Generator forward-loss-backward-update
        z = Variable(torch.randn(mb_size, Z_dim))
        G_sample = G(z, X)
        X_fake = torch.cat([G_sample, X], 0)
        D_fake = D(X_fake) #Here we need to give Xi along with Xg(i.e. Xg+X or G_sample+X)

        G_loss = binary_cross_entropy(D_fake, ones_label_fake)

        G_loss.backward()
        G_solver.step()

        # Housekeeping - reset gradient
        D.zero_grad()

        #Print and plot every now and then
        #if it % 2 == 0:

        print('Iter-{}; D_loss: {}; G_loss: {}'.format(it, D_loss.data.numpy(), G_loss.data.numpy()))



        it+= batch_size
           

epoch: 0
Iter-0; D_loss: [ 1.3923099]; G_loss: [ 0.68064016]


/home/satishk/miniconda3/envs/lre17/lib/python3.6/site-packages/torch/nn/functional.py:1168: UserWarning: Using a target size (torch.Size([256])) that is different to the input size (torch.Size([256, 1])) is deprecated. Please ensure they have the same size.
  "Please ensure they have the same size.".format(target.size(), input.size()))
/home/satishk/miniconda3/envs/lre17/lib/python3.6/site-packages/torch/nn/functional.py:1168: UserWarning: Using a target size (torch.Size([512])) that is different to the input size (torch.Size([512, 1])) is deprecated. Please ensure they have the same size.
  "Please ensure they have the same size.".format(target.size(), input.size()))


Iter-256; D_loss: [ 1.32532799]; G_loss: [ 0.67773587]
Iter-512; D_loss: [ 1.26042986]; G_loss: [ 0.67278099]
Iter-768; D_loss: [ 1.20539045]; G_loss: [ 0.67248237]
Iter-1024; D_loss: [ 1.1521492]; G_loss: [ 0.67095649]
Iter-1280; D_loss: [ 1.10531056]; G_loss: [ 0.67292452]
Iter-1536; D_loss: [ 1.05051136]; G_loss: [ 0.66994667]
Iter-1792; D_loss: [ 1.00457585]; G_loss: [ 0.67301136]
Iter-2048; D_loss: [ 0.96177876]; G_loss: [ 0.67975926]
Iter-2304; D_loss: [ 0.91761458]; G_loss: [ 0.68670934]
Iter-2560; D_loss: [ 0.87502897]; G_loss: [ 0.69697028]
Iter-2816; D_loss: [ 0.82520777]; G_loss: [ 0.70500743]
Iter-3072; D_loss: [ 0.77675295]; G_loss: [ 0.71681732]
Iter-3328; D_loss: [ 0.73848492]; G_loss: [ 0.7353968]
Iter-3584; D_loss: [ 0.68780065]; G_loss: [ 0.74879307]
epoch: 1
Iter-0; D_loss: [ 0.65560472]; G_loss: [ 0.77047747]
Iter-256; D_loss: [ 0.61212242]; G_loss: [ 0.78551167]
Iter-512; D_loss: [ 0.56940925]; G_loss: [ 0.80017954]
Iter-768; D_loss: [ 0.54405999]; G_loss: [ 0.8227

Iter-3584; D_loss: [ 0.03946702]; G_loss: [ 1.69838524]
epoch: 10
Iter-0; D_loss: [ 0.03956528]; G_loss: [ 1.70989585]
Iter-256; D_loss: [ 0.03854638]; G_loss: [ 1.72051835]
Iter-512; D_loss: [ 0.0376025]; G_loss: [ 1.73124337]
Iter-768; D_loss: [ 0.03661457]; G_loss: [ 1.74178374]
Iter-1024; D_loss: [ 0.03613324]; G_loss: [ 1.75250053]
Iter-1280; D_loss: [ 0.03557245]; G_loss: [ 1.76296735]
Iter-1536; D_loss: [ 0.03435844]; G_loss: [ 1.77306628]
Iter-1792; D_loss: [ 0.03376666]; G_loss: [ 1.78340316]
Iter-2048; D_loss: [ 0.0349505]; G_loss: [ 1.79435766]
Iter-2304; D_loss: [ 0.03302854]; G_loss: [ 1.80368519]
Iter-2560; D_loss: [ 0.03306772]; G_loss: [ 1.81399095]
Iter-2816; D_loss: [ 0.03119673]; G_loss: [ 1.8233062]
Iter-3072; D_loss: [ 0.03064796]; G_loss: [ 1.83322942]
Iter-3328; D_loss: [ 0.03007655]; G_loss: [ 1.84303677]
Iter-3584; D_loss: [ 0.02909223]; G_loss: [ 1.85258937]
epoch: 11
Iter-0; D_loss: [ 0.02926257]; G_loss: [ 1.86254239]
Iter-256; D_loss: [ 0.02861589]; G_loss:

Iter-3072; D_loss: [ 0.00570022]; G_loss: [ 2.69173694]
Iter-3328; D_loss: [ 0.00563192]; G_loss: [ 2.6960845]
Iter-3584; D_loss: [ 0.00553396]; G_loss: [ 2.70036173]
epoch: 20
Iter-0; D_loss: [ 0.00559557]; G_loss: [ 2.70470476]
Iter-256; D_loss: [ 0.00556289]; G_loss: [ 2.70895791]
Iter-512; D_loss: [ 0.00552863]; G_loss: [ 2.71313524]
Iter-768; D_loss: [ 0.00539697]; G_loss: [ 2.71733832]
Iter-1024; D_loss: [ 0.00538412]; G_loss: [ 2.72152305]
Iter-1280; D_loss: [ 0.00540677]; G_loss: [ 2.72572374]
Iter-1536; D_loss: [ 0.00526098]; G_loss: [ 2.72984529]
Iter-1792; D_loss: [ 0.00523661]; G_loss: [ 2.73399878]
Iter-2048; D_loss: [ 0.00568078]; G_loss: [ 2.73825979]
Iter-2304; D_loss: [ 0.00525238]; G_loss: [ 2.7421174]
Iter-2560; D_loss: [ 0.00541104]; G_loss: [ 2.74629879]
Iter-2816; D_loss: [ 0.00505239]; G_loss: [ 2.7502284]
Iter-3072; D_loss: [ 0.00504457]; G_loss: [ 2.75433087]
Iter-3328; D_loss: [ 0.00498451]; G_loss: [ 2.75838947]
Iter-3584; D_loss: [ 0.00490368]; G_loss: [ 2.7

...

Iter-512; D_loss: [ 0.00047564]; G_loss: [ 3.96612144]
Iter-768; D_loss: [ 0.00046684]; G_loss: [ 3.96739101]
Iter-1024; D_loss: [ 0.00046504]; G_loss: [ 3.96863246]
Iter-1280; D_loss: [ 0.00047443]; G_loss: [ 3.96990347]
Iter-1536; D_loss: [ 0.00046421]; G_loss: [ 3.97116613]
Iter-1792; D_loss: [ 0.00045956]; G_loss: [ 3.97239923]
Iter-2048; D_loss: [ 0.00051725]; G_loss: [ 3.97364855]
Iter-2304; D_loss: [ 0.00046786]; G_loss: [ 3.97485733]
Iter-2560; D_loss: [ 0.0004934]; G_loss: [ 3.97608447]
Iter-2816; D_loss: [ 0.00045572]; G_loss: [ 3.97734404]
Iter-3072; D_loss: [ 0.00046144]; G_loss: [ 3.97861195]
Iter-3328; D_loss: [ 0.00045498]; G_loss: [ 3.97984529]
Iter-3584; D_loss: [ 0.00045676]; G_loss: [ 3.98108387]
epoch: 60
Iter-0; D_loss: [ 0.00045587]; G_loss: [ 3.98234081]
Iter-256; D_loss: [ 0.00045862]; G_loss: [ 3.98357129]
Iter-512; D_loss: [ 0.00045853]; G_loss: [ 3.98479342]
Iter-768; D_loss: [ 0.0004501]; G_loss: [ 3.98602152]
Iter-1024; D_loss: [ 0.00044832]; G_loss: [ 3.98

Iter-0; D_loss: [ 0.00033588]; G_loss: [ 4.13777399]
Iter-256; D_loss: [ 0.00033817]; G_loss: [ 4.1388402]
Iter-512; D_loss: [ 0.000338]; G_loss: [ 4.13991499]
Iter-768; D_loss: [ 0.00033214]; G_loss: [ 4.14097595]
Iter-1024; D_loss: [ 0.00033059]; G_loss: [ 4.14205742]
Iter-1280; D_loss: [ 0.00033782]; G_loss: [ 4.14311647]
Iter-1536; D_loss: [ 0.00033085]; G_loss: [ 4.14419651]
Iter-1792; D_loss: [ 0.00032676]; G_loss: [ 4.14528036]
Iter-2048; D_loss: [ 0.0003693]; G_loss: [ 4.1463418]
Iter-2304; D_loss: [ 0.00033332]; G_loss: [ 4.14737368]
Iter-2560; D_loss: [ 0.00035216]; G_loss: [ 4.14843512]
Iter-2816; D_loss: [ 0.00032504]; G_loss: [ 4.14950275]
Iter-3072; D_loss: [ 0.00032952]; G_loss: [ 4.1505928]
Iter-3328; D_loss: [ 0.00032471]; G_loss: [ 4.15164709]
Iter-3584; D_loss: [ 0.0003268]; G_loss: [ 4.1527319]
epoch: 70
Iter-0; D_loss: [ 0.00032547]; G_loss: [ 4.1537838]
Iter-256; D_loss: [ 0.0003277]; G_loss: [ 4.15483189]
Iter-512; D_loss: [ 0.00032754]; G_loss: [ 4.15590143]
Ite

Iter-3328; D_loss: [ 0.00024897]; G_loss: [ 4.28678131]
Iter-3584; D_loss: [ 0.00025105]; G_loss: [ 4.28774738]
epoch: 79
Iter-0; D_loss: [ 0.00024962]; G_loss: [ 4.28870869]
Iter-256; D_loss: [ 0.00025147]; G_loss: [ 4.28963041]
Iter-512; D_loss: [ 0.00025127]; G_loss: [ 4.29056501]
Iter-768; D_loss: [ 0.00024719]; G_loss: [ 4.29152346]
Iter-1024; D_loss: [ 0.00024584]; G_loss: [ 4.29244947]
Iter-1280; D_loss: [ 0.00025153]; G_loss: [ 4.29340363]
Iter-1536; D_loss: [ 0.00024655]; G_loss: [ 4.29433203]
Iter-1792; D_loss: [ 0.00024297]; G_loss: [ 4.29526758]
Iter-2048; D_loss: [ 0.00027554]; G_loss: [ 4.29619598]
Iter-2304; D_loss: [ 0.00024828]; G_loss: [ 4.29714298]
Iter-2560; D_loss: [ 0.00026266]; G_loss: [ 4.29807043]
Iter-2816; D_loss: [ 0.00024233]; G_loss: [ 4.29898548]
Iter-3072; D_loss: [ 0.00024591]; G_loss: [ 4.29994965]
Iter-3328; D_loss: [ 0.00024217]; G_loss: [ 4.30088282]
Iter-3584; D_loss: [ 0.00024424]; G_loss: [ 4.30183744]
epoch: 80
Iter-0; D_loss: [ 0.0002428]; G_lo

Iter-2816; D_loss: [ 0.00019138]; G_loss: [ 4.41908312]
Iter-3072; D_loss: [ 0.00019435]; G_loss: [ 4.41993809]
Iter-3328; D_loss: [ 0.00019129]; G_loss: [ 4.42079163]
Iter-3584; D_loss: [ 0.00019325]; G_loss: [ 4.42164755]
epoch: 89
Iter-0; D_loss: [ 0.00019183]; G_loss: [ 4.42250109]
Iter-256; D_loss: [ 0.00019336]; G_loss: [ 4.42334414]
Iter-512; D_loss: [ 0.00019313]; G_loss: [ 4.42416]
Iter-768; D_loss: [ 0.00019018]; G_loss: [ 4.42501163]
Iter-1024; D_loss: [ 0.00018901]; G_loss: [ 4.42585135]
Iter-1280; D_loss: [ 0.00019358]; G_loss: [ 4.42669868]
Iter-1536; D_loss: [ 0.0001899]; G_loss: [ 4.42754841]
Iter-1792; D_loss: [ 0.00018675]; G_loss: [ 4.42837095]
Iter-2048; D_loss: [ 0.00021241]; G_loss: [ 4.42921638]
Iter-2304; D_loss: [ 0.00019112]; G_loss: [ 4.43005753]
Iter-2560; D_loss: [ 0.00020239]; G_loss: [ 4.43088055]
Iter-2816; D_loss: [ 0.00018668]; G_loss: [ 4.43172359]
Iter-3072; D_loss: [ 0.0001896]; G_loss: [ 4.43255663]
Iter-3328; D_loss: [ 0.0001866]; G_loss: [ 4.4334

Iter-2048; D_loss: [ 0.00017174]; G_loss: [ 4.53774214]
Iter-2304; D_loss: [ 0.00015436]; G_loss: [ 4.53850698]
Iter-2560; D_loss: [ 0.00016357]; G_loss: [ 4.53927612]
Iter-2816; D_loss: [ 0.00015087]; G_loss: [ 4.54004717]
Iter-3072; D_loss: [ 0.00015331]; G_loss: [ 4.54079485]
Iter-3328; D_loss: [ 0.0001508]; G_loss: [ 4.5415864]
Iter-3584; D_loss: [ 0.00015261]; G_loss: [ 4.5423708]
epoch: 99
Iter-0; D_loss: [ 0.00015127]; G_loss: [ 4.54314518]
Iter-256; D_loss: [ 0.00015254]; G_loss: [ 4.54388952]
Iter-512; D_loss: [ 0.00015231]; G_loss: [ 4.54465199]
Iter-768; D_loss: [ 0.00015011]; G_loss: [ 4.54542255]
Iter-1024; D_loss: [ 0.00014907]; G_loss: [ 4.54619789]
Iter-1280; D_loss: [ 0.00015283]; G_loss: [ 4.54696131]
Iter-1536; D_loss: [ 0.00015002]; G_loss: [ 4.54773283]
Iter-1792; D_loss: [ 0.00014725]; G_loss: [ 4.5484767]
Iter-2048; D_loss: [ 0.00016791]; G_loss: [ 4.54922867]
Iter-2304; D_loss: [ 0.00015091]; G_loss: [ 4.54999352]
Iter-2560; D_loss: [ 0.00015992]; G_loss: [ 4.55

In [51]:
df_SF=pd.DataFrame()
samples_per_class = 1000
#c = np.zeros(shape=[samples_per_class, y_dim], dtype='float32')
#c[:, np.random.randint(0, 10)] = 1.
for i in range(14):
    #print(i)
    c = np.zeros(shape=[samples_per_class, y_dim], dtype='float32')
    c[:, i] = 1.
    c_df=pd.DataFrame(c)
    df_SF = df_SF.append(c_df,ignore_index = True)
    
    

In [ ]:
c_df.shape

In [ ]:
df_SF.shape[0]

In [ ]:
c_gen = df_SF.values

In [ ]:
z_gen = Variable(torch.randn(df_SF.shape[0], Z_dim))

In [ ]:
z_gen.data.shape

In [ ]:
c_gen = Variable(torch.from_numpy(c_gen))

In [ ]:
samples = G(z_gen, c_gen)

In [ ]:
samples.data.shape

In [ ]:
#Taking the generated iVectors we will try to check the acc by MLP

In [ ]:
train_X1 = samples.data.numpy()

In [ ]:
type(train_X1)

In [ ]:
train_y1 = c_gen.data.numpy()

In [ ]:
type(train_y1)

In [ ]:
train_y1

In [ ]:
X_train = pd.DataFrame(X_train)
Y_train = pd.DataFrame(Y_train)


In [ ]:
train_X1 = pd.DataFrame(train_X1)
train_y1 = pd.DataFrame(train_y1)

In [ ]:
#train_y

In [ ]:
train_X = X_train.append(train_X1, ignore_index=True)
train_y = Y_train.append(train_y1, ignore_index=True)

In [ ]:
#train_X,  train_y = shuffle(train_X, train_y, random_state=0)

In [ ]:
train_X = train_X.values
train_y = train_y.values

In [ ]:
train_y

In [ ]:
model = Sequential()
model.add(Dense(512, input_dim=500))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(nb_classes))
model.add(Activation('softmax'))

In [ ]:
model.summary()

In [ ]:
adam=optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
best_weights_filepath = '/home/satishk/saved_weights/best_weights_2l_MLP_11.hdf5'
saveBestModel = ModelCheckpoint(best_weights_filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='auto')


In [ ]:
batch_size = 256
nb_epoch=30

In [ ]:
#Checking Accuracy with training+augmented data train_X and train_y are 'train + augmented' data
history = model.fit(train_X, train_y, batch_size=batch_size, epochs=nb_epoch,verbose=2, 
                    validation_data=(X_test , y_test),callbacks=[saveBestModel])

In [ ]:
#Frame label accuracy
score = model.evaluate(X_test, y_test, verbose=0)
print('ERROR after Data Augmentation %:', 1-score[1])